Download dataset from TTS 

In [ ]:
import os
#from TTS.utils.downloaders import download_vctk

output_path = os.getcwd()
dataset_path = os.path.join(output_path, "../VCTK/")

#download_vctk(dataset_path)

In [1]:
import numpy as np 
print(np.__version__)

1.26.4


In [5]:
from pathlib import Path
from encoder.preprocess import preprocess_vctk
import numpy as np
import torch
from synthesizer.inference import Synthesizer


from vocoder.inference import infer_waveform,load_model



In [ ]:
# pre-processing dataset 

datasets_root = Path(".")  
out_dir = Path("VCTK/encoder")  

# Create the output directory if it doesn't exist
out_dir.mkdir(parents=True, exist_ok=True)

# Run preprocessing
preprocess_vctk(datasets_root, out_dir, skip_existing=False)



In [12]:
from encoder.inference import embed_utterance, load_model

# Encoder model load 
# encoder2.pt is the trained weight provided by github https://github.com/CorentinJ/Real-Time-Voice-Cloning/wiki/Pretrained-models
from pathlib import Path

base_path = Path.cwd()  # gets current working dir where the notebook runs
weights_fpath = base_path/ "saved_models" / "default" / "encoder2.pt"

load_model(weights_fpath=weights_fpath, device="cuda")


Loaded encoder "encoder2.pt" trained to step 1564501


In [13]:
from encoder.audio import preprocess_wav
#transforming a new speaker audio and generate utterance 
audio_file = base_path/ "VCTK" / "testing_audio.wav"
file = preprocess_wav(audio_file)
utterances = embed_utterance(file)

In [14]:
# Loading Synthesizer model 
synthesizer_weight = base_path/ "saved_models"/"default"/"synthesizer.pt"
model2 = Synthesizer(model_fpath=synthesizer_weight,verbose=False)

In [15]:
# giving new text & the new speaker utterance to the synthesize model to generate spectrogram
new_text = ['Romeo, take me somewhere we can be alone  I be waiting, all that has left to do is run  You be the prince and I will be the princess  It is a love story, baby, just say, Yes']
syn_model = model2.synthesize_spectrograms(new_text,embeddings=utterances)

Trainable Parameters: 30.870M


In [16]:
print(len(syn_model[0]))

80


In [19]:
# loading the vocoder model & generate the waveform for the new speaker 
from vocoder.inference import load_model
vocoder_weight = base_path/ "saved_models"/"default"/"vocoder.pt"
load_model(weights_fpath= vocoder_weight)
generated_wave = infer_waveform(syn_model[0],normalize=False)

Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /home/ngtra14/DLProject/saved_models/default/vocoder.pt
{| ████████████████ 437000/441600 | Batch Size: 46 | Gen Rate: 51.7kHz | }

In [20]:
import sounddevice as sd
print(sd.query_devices())
try:
    sd.stop()
    sd.play(generated_wave, Synthesizer.sample_rate)
except sd.PortAudioError as e:
    print("\nCaught exception: %s" % repr(e))




Caught exception: PortAudioError('Error querying device -1')


If Sounddevice does not work, use below 

In [21]:
from scipy.io.wavfile import write
import numpy as np
waveform_int16 = np.int16(generated_wave / np.max(np.abs(generated_wave)) * 32767)
write("output.wav", Synthesizer.sample_rate, waveform_int16)

print("Saved synthesized audio as output.wav")

Saved synthesized audio as output.wav
